In [ ]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd /content/Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 75 (delta 33), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (75/75), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     |████████████████████████████████| 19.4MB 167kB/s 
     |████████████████████████████████| 92kB 13.5MB/s 
     |████████████████████████████████| 460kB 58.6MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2020-12-14 16:34:02--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz

In [ ]:
import MeCab
import re
import random
import time
import datetime
import numpy as np      
import pandas as pd       

import matplotlib.pyplot as plt   
from IPython.display import Image
from collections import Counter
from tqdm import tqdm 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier

In [ ]:
mecab = MeCab.Tagger()

def mecab_morphs(text):
    morphs = []
    pattern = re.compile(".*\t[A-Z]+") 
    # konlpy의 mecab output 형태와 같게 만들어주기
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
    
    for token in temp:
        morphs.append(token[0])
    return morphs

In [ ]:
%%time
train = pd.read_csv("/content/drive/My Drive/dacon/news_train.csv") # train.csv 불러오기
test = pd.read_csv("/content/drive/My Drive/dacon/news_test.csv") # test.csv 불러오기
train["id"] = train["n_id"].astype(str) + '_' + train["ord"].astype(str)
train.drop(['n_id', 'ord'],axis = 1) 
print(test.shape)
print(train.shape)


(142565, 6)
(118745, 7)
CPU times: user 694 ms, sys: 104 ms, total: 798 ms
Wall time: 5.94 s


In [ ]:
%%time
def preprocess(text):
    # remove hyperlinks
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    text = re.sub(r'http?:\/\/.*[\r\n]*', '', text)
    #remove some puncts (except . ! ?)
    text=re.sub(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '',text)
    text=" ".join(text.split())
    return text

train['clean_text'] = train['content'].apply(preprocess)
test['clean_text'] = test['content'].apply(preprocess)
test.head()

CPU times: user 1.07 s, sys: 17.1 ms, total: 1.09 s
Wall time: 1.09 s


In [ ]:
train['info'].value_counts()


0    71813
1    46932
Name: info, dtype: int64

In [ ]:
%%time
random.seed(2020) 
train_nsm_list=list(train[train['info']!=1].index)
train_nsmishing=random.sample(train_nsm_list, 71000) 
random.seed(2020)
train_sm_list=list(train[train['info']==1].index)
train_smishing=random.sample(train_sm_list, 46932)
train_xx=train.iloc[train_smishing+train_nsmishing,:].reset_index(drop=True)

train_yy=pd.DataFrame(train['info'],columns=['info']) 
train_yyy=train_yy.iloc[train_smishing+train_nsmishing,:].reset_index(drop=True)
test['info']=2
test_xx=pd.DataFrame(test['clean_text'])
test_yyy=pd.DataFrame(test['info'])

CPU times: user 220 ms, sys: 1.53 ms, total: 222 ms
Wall time: 227 ms


In [ ]:
train_doc = [(mecab_morphs(x), y) for x, y in tqdm(zip( train_xx['clean_text'], train_yyy['info']))]
test_doc = [(mecab_morphs(x), y) for x, y in tqdm(zip( test_xx['clean_text'], test_yyy['info']))]


117932it [00:12, 9442.77it/s]
142565it [00:14, 9606.51it/s] 


In [ ]:
%%time
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도','을', '를', '으로', '자', '에', '와', '한', '하다']
def get_model_input(_words): #필요없는 단어들 없애는 함수
    global stopwords
    _words = [x for x in _words if x[0] not in stopwords]
    _words = [x for x in _words if x[:-1] not in stopwords]
    for i in range(len(_words)-1):
        yield _words[i]

X_train, Y_train = [], []
for lwords in train_doc:
    Y_train.append(lwords[1])
    
    temp = []
    for x in get_model_input(lwords[0]):
        if len(x) != 1:
            temp.append("{}".format(x))
    X_train.append(" ".join(temp))

X_test = []
for lwords in test_doc:
    
    temp = []
    for x in get_model_input(lwords[0]):
        if len(x) != 1:
            temp.append("{}".format(x))
    X_test.append(" ".join(temp))

CPU times: user 4.71 s, sys: 19.6 ms, total: 4.73 s
Wall time: 4.74 s


In [ ]:
%%time
#Tfidvectorizing mecab-preprocessed word for modelling
vectorizer=TfidfVectorizer(ngram_range=(1, 3),
    min_df=2,   
    max_features=10000,
    sublinear_tf=True,
    lowercase=False,
    use_idf=True)

vec_x_train= vectorizer.fit_transform(X_train)
vec_x_test= vectorizer.transform(X_test)

CPU times: user 8.75 s, sys: 163 ms, total: 8.91 s
Wall time: 8.93 s


In [ ]:
%%time
#modelling with fine-tuned parameter
lgbm = LGBMClassifier(n_estimators=380,
                     learning_rate=0.035,
                     max_depth=7,
                     min_child_samples=50,
                     random_state=2020)
lgbm.fit(vec_x_train, Y_train)

CPU times: user 42.1 s, sys: 184 ms, total: 42.2 s
Wall time: 21.5 s


In [ ]:
%%time
#prediction
y_train_pred1=lgbm.predict_proba(vec_x_train)
y_train_pred1_one= [1 if i[1] >= 0.5 else 0 for i in y_train_pred1]

y_test_pred1=lgbm.predict_proba(vec_x_test)
y_test_pred1_one= [1 if i[1] >= 0.5 else 0 for i in y_test_pred1]

CPU times: user 4.95 s, sys: 8.57 ms, total: 4.96 s
Wall time: 2.58 s


In [ ]:
%%time
submission = pd.DataFrame(
{"id" : test.id,
"info" : y_test_pred1_one})
submission.head()
submission.to_csv("mecab_fast_lightGBM.csv", index = False)

CPU times: user 174 ms, sys: 4.98 ms, total: 179 ms
Wall time: 180 ms
